Pipeline HEROHE

In [ ]:
# Libraries import
!curl -s https://course.fast.ai/setup/colab | bash

from fastai import *
from fastai.tabular import * 
import numpy as np
from google.colab import files
from zipfile import ZipFile
from fastai.utils import * 
from google.colab import drive
drive.mount('/content/gdrive')
!ls gdrive

In [ ]:
# Copy CSVs from Drive
!cp  '''insert the path to the train set here''' train_valid.csv 
!cp '''insert the path to the test set here''' test.csv
!cp '''insert the path to the ground truth set here''' grdTruth.csv

In [ ]:
# Generating DataFrame
df = pd.read_csv('train_valid.csv')
test_df = pd.read_csv('test.csv')
grd = pd.read_csv('grdTruth.csv')
df

In [ ]:
#
procs = [FillMissing, Categorify, Normalize]
valid_idx = range(len(df)-68, len(df))
dep_var = 'truth'
path = '.'
data = TabularDataBunch.from_df(path, df, dep_var, valid_idx=valid_idx, procs=procs)#, bs=16)
print(data.train_ds.cont_names)

In [ ]:
# Metrics load
fbeta = FBeta()
auroc = AUROC()

In [ ]:
# Tabular learner setup
learn = tabular_learner(data, layers = [500, 250, 50], metrics = [accuracy, fbeta, auroc])
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
# Learning process setup
learn.fit_one_cycle(6, 1e-04)

In [ ]:
# Save model locally
learn.save('tabular')

In [ ]:
# Save model on Drive
!cp 'models/tabular.pth' '''insert the path to model saving location'''

In [ ]:
# Loading model from Drive
!cp '''insert the path to the model here''' models/tabular.pth

In [ ]:
her2_model = load_learner("models", "tabular.pth", test=temp_df)

**Testing**

In [ ]:
# Preliminary check
r = range(0,len(test_df))
l = []
for count in r:
  l.append(learn.predict(test_df.iloc[count])[0])
print(l)

temp_df = test_df
temp_df["truth"] = l
temp_df = temp_df[["caseID", "truth"]]

check = temp_df.merge(grd, on = "caseID", how = "inner")
check.truth = check.truth.astype(int)
check.dtypes

In [ ]:
# Identification of true_positive, true_negative, false_positive and false_negative 
truth = check["truth"]
real = check["real"]
slide = check["caseID"]

true_pos = 0
true_neg = 0
false_pos = 0 
false_neg = 0

for i in range(0, len(truth)):
  if(truth[i] == real[i]):
    if(truth[i] == 1):
        stringa = "Correctly predicted positive"
        true_pos += 1
    else:
      stringa = "Correctly predicted negative"
      true_neg += 1
  else:
    if(real[i] == 1):
      stringa = "Predicted false negative"
      false_neg += 1
    else:
      stringa = "Predicted false positive"
      false_pos +=1
  print(f"Slide {slide[i]}. {stringa}. ({truth[i]}, {real[i]})")

print(f"\n\nTrue Positive: {true_pos}\nTrue Negative: {true_neg}\n")
print(f"False Positive: {false_pos}\nFalse Negative: {false_neg}\n")


In [ ]:
# Test metrics computation
total = true_pos + true_neg + false_pos + false_neg
print("Metrics on Test Set")
print(f"Accuracy: {round((true_pos + true_neg) / total, 3)}")
print(f"True Positive Rate: {round(true_pos / (false_pos + true_pos), 3)}")
print(f"False Positive Rate: {round(false_pos / (false_pos + true_neg), 3)}")
precision = round((true_pos / (true_pos + false_pos)), 3)
print(f"Precision: {precision}")
recall = round((true_pos / (true_pos + false_neg)), 3)
print(f"Recall: {recall}")
print(f"F1: {round(2 * (1 / ((1/precision) + (1/recall))), 3)}")